### Import Dependencies

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, MatchAny, FieldCondition, Filter, Prefetch, FusionQuery

import pandas as pd
import numpy as np
import openai
import json
import tiktoken

/Users/ranjan/project/swirl-ai/03-ai-engineering-bootcamp-cohort-3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Retrieve all item IDs from Amazon Items Qdrant Collection

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
dummy_vector = np.zeros(1536).tolist()

In [4]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [5]:
payload.points

[ScoredPoint(id=130, version=3, score=0.0, payload={'parent_asin': 'B09XD4SSW6'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=381, version=3, score=0.0, payload={'parent_asin': 'B09QPM7YWT'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=173, version=3, score=0.0, payload={'parent_asin': 'B0BKL58DBY'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=461, version=3, score=0.0, payload={'parent_asin': 'B0B5HW7WTN'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=58, version=3, score=0.0, payload={'parent_asin': 'B0B9YLN7T5'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=405, version=3, score=0.0, payload={'parent_asin': 'B09ZLD899T'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=192, version=3, score=0.0, payload={'parent_asin': 'B0B3JSQRPK'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=227, version=3, score=0.0, payload={'parent_asin': 'B09QMSRT2Q'}, vector=No

In [6]:
len(payload.points)

1000

In [7]:
parent_asin_list = [item.payload["parent_asin"] for item in payload.points]

In [8]:
parent_asin_list

['B09XD4SSW6',
 'B09QPM7YWT',
 'B0BKL58DBY',
 'B0B5HW7WTN',
 'B0B9YLN7T5',
 'B09ZLD899T',
 'B0B3JSQRPK',
 'B09QMSRT2Q',
 'B09SD4LWP4',
 'B09YXWWSGC',
 'B0BLYZC947',
 'B09ZMHZFLH',
 'B0BWTV359C',
 'B00K9TK7RI',
 'B0BDSWTT57',
 'B0BMTKDPPN',
 'B09Y1K339K',
 'B0BYCKWYKV',
 'B0B1T59VHW',
 'B0CDX1S8J2',
 'B0BYZHMVXP',
 'B0CBWZJGKZ',
 'B0B8V7CN6X',
 'B09P4ZNKYV',
 'B0B4JYZMT3',
 'B0C996WY16',
 'B0C6CNNBG9',
 'B0C9DBV55F',
 'B0BBLNH6F2',
 'B0C52WNWY6',
 'B09X1B9KTF',
 'B0BB8XCJ61',
 'B0B2W2FKLC',
 'B092Z9CTNK',
 'B09ZDFZQZ5',
 'B08ZSBSJHY',
 'B09TW58RG4',
 'B0C8TK3VV3',
 'B0C55LMF82',
 'B09ZW8HKWG',
 'B0BMQ1SD35',
 'B0C16R1TW8',
 'B0BXKHLT54',
 'B0C8HG9Q6J',
 'B0BPCNM7X8',
 'B09VBRWMJ5',
 'B0BZVG1HV4',
 'B0BZW3VJ2T',
 'B0B3T827XP',
 'B09TPPL2Q1',
 'B0B93N2LW8',
 'B09SH4R8VV',
 'B0BJPF3KR5',
 'B0B7D63729',
 'B0B2RKB1HP',
 'B09S6LXSGL',
 'B0C373P4M7',
 'B0C32VNY83',
 'B09V6NVNRG',
 'B0C53GR767',
 'B0BJDRZLM2',
 'B0CFHLHYY7',
 'B08CK26MTZ',
 'B09ZLFV5PC',
 'B0BLY789SC',
 'B0C3J1B5J2',
 'B0BWVD7K

#### Load Amazon Reviews Dataset

In [9]:
df_reviews = pd.read_json("../../data/Electornics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [50]:
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Perfect!,This is perfect! Thank you so much!!! I absolu...,[],B09992M2LX,B09ZPV8WBV,AHX4XWVVQUKT3FCNWCVASDF4Q56Q,2022-08-05 04:06:39.589,0,True
1,5,3ft mini usb cables,I don't have many things that still use a mini...,[],B09Y94B2NM,B09Y95BMKX,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-07-16 16:03:28.714,3,True
2,5,I would buy it again.,Great product. Worked well for what we needed ...,[],B07T55DL33,B0B2JWCMCY,AF5KFHNT3TQJ2GNSE3FCDFQOBICA,2019-12-09 22:35:00.531,0,True
3,5,Great to Have Around,My husband and I were recently working a booth...,[],B09M89JN7B,B0BYYGZHG5,AHV6QCNBJNSGLATP56JAWJ3C4G2A,2022-03-22 01:43:49.342,0,False
4,5,Easy to use,Work as advertised and at a very good price.,[],B07T55DL33,B0B2JWCMCY,AG7WKTZINOFIXMZJYIPKIB7PV7NQ,2019-12-28 06:12:24.960,0,True


In [11]:
len(df_reviews)

299618

In [12]:
df_reviews_sample = df_reviews[df_reviews["parent_asin"].isin(parent_asin_list)]

In [13]:
len(df_reviews_sample)

200598

#### Define functions to preprocess reviews data

In [14]:
def preprocess_reviews_data(row):
    return f"{row['title']} {row['text']}"

In [15]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

In [16]:
encoding.encode("Can I get some earphones")

[6854, 358, 636, 1063, 2487, 17144]

In [17]:
def token_count(row, model="text-embedding-3-small"):

    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(row["preprocessed_data"]))

In [18]:
df_reviews_sample["preprocessed_data"] = df_reviews_sample.apply(preprocess_reviews_data, axis=1)

/var/folders/cm/_ps_xv6x60bdskprtj58xn0c0000gn/T/ipykernel_13155/1882656165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_sample["preprocessed_data"] = df_reviews_sample.apply(preprocess_reviews_data, axis=1)


In [19]:
df_reviews_sample["preprocessed_data_token_count"] = df_reviews_sample.apply(token_count, axis=1)

/var/folders/cm/_ps_xv6x60bdskprtj58xn0c0000gn/T/ipykernel_13155/1530805122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_sample["preprocessed_data_token_count"] = df_reviews_sample.apply(token_count, axis=1)


In [ ]:
df_reviews_sample.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,preprocessed_data,preprocessed_data_token_count


In [21]:
len(df_reviews_sample)

200598

In [46]:
df_reviews_sample = df_reviews_sample[df_reviews_sample["preprocessed_data_token_count"] < 8192]

In [47]:
len(df_reviews_sample)

200598

In [24]:
total_tokens = df_reviews_sample["preprocessed_data_token_count"].sum()

In [25]:
total_tokens

np.int64(12097767)

#### Create a new Qdrant collection for Reviews

In [26]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-01-reviews",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [27]:
qdrant_client.create_payload_index(
    collection_name="Amazon-items-collection-01-reviews",
    field_name="parent_asin",
    field_schema=PayloadSchemaType.KEYWORD
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

#### Embedding Functions

In [28]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [29]:
def get_embeddings_batch(text_list, model="text-embedding-3-small", batch_size=100):
    
    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input=text_list, model=model)
        return [embedding.embedding for embedding in response.data]
    
    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed {counter * batch_size} of {len(text_list)}")
        counter += 1
    
    return all_embeddings

#### Embed the text and add additional fields to the payload of each vector for reviews

In [30]:
data_to_embed_reviews = df_reviews_sample[["preprocessed_data", "parent_asin"]].to_dict(orient="records")

In [ ]:
data_to_embed_reviews

[{'preprocessed_data': 'Four Stars Perfect', 'parent_asin': 'B08ZSBSJHY'},
 {'preprocessed_data': 'Five Stars Worked perfect and was no problem to set up the drivers. Well worth the money.',
  'parent_asin': 'B0BJ1SLJH1'},
 {'preprocessed_data': 'Works Fine This card is very small, I was surprised when I saw it.  It uses a Realtek chip so windows had no problem recognizing it and I expect it to last for as long as I need it.',
  'parent_asin': 'B0BJ1SLJH1'},
 {'preprocessed_data': 'Great for velcroing LARGE things! First, note that this isn\'t just a hoop-and-loop product. This is VELCRO, the brand name itself! So you know the expectations are set high. And they deliver.<br />I\'ve owned similar products that are good for wrapping up cables I bring to jobs, but when I\'m doing a live stream event and need to run hundreds of feet of CAT5 cable, you need a BIG strap to secure them up when wrapped up to go. These 30" straps are not only big, but they are stretchy as well, so I could easil

In [32]:
text_to_embed_reviews = [data["preprocessed_data"] for data in data_to_embed_reviews]

In [33]:
text_to_embed_reviews

['Four Stars Perfect',
 'Five Stars Worked perfect and was no problem to set up the drivers. Well worth the money.',
 'Works Fine This card is very small, I was surprised when I saw it.  It uses a Realtek chip so windows had no problem recognizing it and I expect it to last for as long as I need it.',
 'Great for velcroing LARGE things! First, note that this isn\'t just a hoop-and-loop product. This is VELCRO, the brand name itself! So you know the expectations are set high. And they deliver.<br />I\'ve owned similar products that are good for wrapping up cables I bring to jobs, but when I\'m doing a live stream event and need to run hundreds of feet of CAT5 cable, you need a BIG strap to secure them up when wrapped up to go. These 30" straps are not only big, but they are stretchy as well, so I could easily make a bundle of very thick cable and hold it all together like a bow. This is exactly what I wanted and needed, thank you Velcro!',
 'Huge Love it. Big and wide. Great clarity. I 

In [34]:
embeddings_reviews = get_embeddings_batch(text_to_embed_reviews, batch_size=500)

Processed 500 of 200598
Processed 1000 of 200598
Processed 1500 of 200598
Processed 2000 of 200598
Processed 2500 of 200598
Processed 3000 of 200598
Processed 3500 of 200598
Processed 4000 of 200598
Processed 4500 of 200598
Processed 5000 of 200598
Processed 5500 of 200598
Processed 6000 of 200598
Processed 6500 of 200598
Processed 7000 of 200598
Processed 7500 of 200598
Processed 8000 of 200598
Processed 8500 of 200598
Processed 9000 of 200598
Processed 9500 of 200598
Processed 10000 of 200598
Processed 10500 of 200598
Processed 11000 of 200598
Processed 11500 of 200598
Processed 12000 of 200598
Processed 12500 of 200598
Processed 13000 of 200598
Processed 13500 of 200598
Processed 14000 of 200598
Processed 14500 of 200598
Processed 15000 of 200598
Processed 15500 of 200598
Processed 16000 of 200598
Processed 16500 of 200598
Processed 17000 of 200598
Processed 17500 of 200598
Processed 18000 of 200598
Processed 18500 of 200598
Processed 19000 of 200598
Processed 19500 of 200598
Proces

In [35]:
len(embeddings_reviews)

200598

In [36]:
pointstructs = []
i = 1
for embedding, data in zip(embeddings_reviews, data_to_embed_reviews):
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload={
                "text": data["preprocessed_data"],
                "parent_asin": data["parent_asin"],
            }
        )
    )
    i += 1

In [37]:
batch_size_qdrant = 100
counter = 1
for i in range(0, len(pointstructs), batch_size_qdrant):
    batch = pointstructs[i:i + batch_size_qdrant]
    qdrant_client.upsert(
        collection_name="Amazon-items-collection-01-reviews",
        wait=True,
        points=batch
    )
    print(f"Processed {counter * batch_size_qdrant} of {len(pointstructs)}")
    counter += 1

Processed 100 of 200598
Processed 200 of 200598
Processed 300 of 200598
Processed 400 of 200598
Processed 500 of 200598
Processed 600 of 200598
Processed 700 of 200598
Processed 800 of 200598
Processed 900 of 200598
Processed 1000 of 200598
Processed 1100 of 200598
Processed 1200 of 200598
Processed 1300 of 200598
Processed 1400 of 200598
Processed 1500 of 200598
Processed 1600 of 200598
Processed 1700 of 200598
Processed 1800 of 200598
Processed 1900 of 200598
Processed 2000 of 200598
Processed 2100 of 200598
Processed 2200 of 200598
Processed 2300 of 200598
Processed 2400 of 200598
Processed 2500 of 200598
Processed 2600 of 200598
Processed 2700 of 200598
Processed 2800 of 200598
Processed 2900 of 200598
Processed 3000 of 200598
Processed 3100 of 200598
Processed 3200 of 200598
Processed 3300 of 200598
Processed 3400 of 200598
Processed 3500 of 200598
Processed 3600 of 200598
Processed 3700 of 200598
Processed 3800 of 200598
Processed 3900 of 200598
Processed 4000 of 200598
Processed

#### A function to run search against reviews on a prefiltered set of product IDs

In [38]:
def retrieve_prefiltered_reviews_data(query, parent_asins, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-reviews",
        prefetch=[
            Prefetch(
                query=query_embedding,
                filter=Filter(
                    must=[
                        FieldCondition(
                            key="parent_asin",
                            match=MatchAny(
                                any=parent_asins
                            )
                        )
                    ]
                ),
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k
    )

    return results

In [39]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B09VDNKL4G"], k=5)

In [40]:
reviews.points

[]

In [41]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B09VDNKL4G", "B09YD1FZDN"], k=5)

In [42]:
reviews.points

[]

In [43]:
reviews = retrieve_prefiltered_reviews_data("bad quality", ["B09VDNKL4G", "B09YD1FZDN"], k=20)

In [44]:
reviews.points

[]

In [45]:
for point in reviews.points:
    print(point.payload["parent_asin"])
    print(point.payload["text"])
    print("-"*100)